# Energy Consumption Forecasting
***
## Table of Contents
***

In [1]:
import torch
import os
import random
from torch import nn
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import numpy as np
from typing import List
import pandas as pd
import seaborn as sns
from pathlib import Path
from torch.utils.data import Dataset, DataLoader, Subset
import matplotlib.pyplot as plt

## 1. Introduction

## 2. Device Agnostic-Code
Mac GPU acceleration (`mps` backend) delivers significant speed-up over CPU for deep learning tasks, especially for large models and batch sizes. On Windows, `cuda` is used instead of `mps`.

In [ ]:
# Set device
# device = "cuda" if torch.cuda.is_available() else "cpu"  # For Windows
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")  # For Mac
device

device(type='mps')

## 3. Loading Dataset
Retrieved from [Kaggle - Hourly Energy Consumption](https://www.kaggle.com/datasets/robikscube/hourly-energy-consumption)

In [3]:
df = pd.read_csv('_datasets/energy_consumption_pjm/AEP_hourly.csv')
df.head()

,Datetime,AEP_MW
0,2004-12-31 01:00:00,13478.0
1,2004-12-31 02:00:00,12865.0
2,2004-12-31 03:00:00,12577.0
3,2004-12-31 04:00:00,12517.0
4,2004-12-31 05:00:00,12670.0
